In [ ]:
%tensorflow_version 1.4
import tensorflow
print(tensorflow.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.4`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM , Bidirectional
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
f = open('/content/drive/My Drive/unbiased_project/HHH.txt', 'r')
newtok = f.readlines()
f.close()

In [ ]:
tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(newtok)

In [ ]:
from keras.models import load_model
model_dwn = load_model('/content/drive/My Drive/unbiased_project/HHH.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [ ]:
sample_texts = ["you are stupid"]
encoded_sample = tokenizer.texts_to_sequences(sample_texts)
# defining a max size for padding.
max_len = 150
# padding the vectors of each datapoint to fixed length of 600.
pad_sample = pad_sequences(encoded_sample,maxlen = max_len,padding='post')
preds = model_dwn.predict(pad_sample)
print(preds)

[[0.99986076]]


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/unbiased_project/train.csv")
df.tail(40)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
1804834,6333864,0.000000,Now post as Thuggernaut.\n\n🙄,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:38:38.347874+00,54,6333196.0,399258,approved,0,0,0,0,0,0.000000,0,4
1804835,6333865,0.000000,Evolution and your account of the origins of t...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:40:09.416494+00,13,6333659.0,398655,approved,0,0,0,0,0,0.000000,0,4
1804836,6333868,0.195652,Master ahole is what this guy was. Putting re...,0.000000,0.108696,0.000000,0.152174,0.065217,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,2017-11-11 00:39:41.960571+00,55,NaN,399503,approved,0,0,0,0,0,0.065217,6,46
1804837,6333869,0.000000,Agreed Bill. Most folks think that it's a myth...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:40:06.695446+00,54,6333578.0,399472,approved,0,0,0,0,0,0.000000,0,4
1804838,6333871,0.700000,Nobody really cares that she supports the scie...,0.000000,0.100000,0.200000,0.700000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:40:22.395923+00,54,NaN,399528,approved,0,0,0,0,0,0.000000,0,10
1804839,6333872,0.200000,I don't know that abortion played much of a pa...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,2017-11-11 00:41:07.880299+00,53,6332177.0,399276,approved,0,0,0,0,0,0.000000,4,5
1804840,6333873,0.000000,Seems like a symptom of Toronto's escalating l...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:41:08.821674+00,54,NaN,399102,approved,0,0,0,0,0,0.000000,0,4
1804841,6333875,0.166667,"""It doesn't matter when it's erected and for w...",0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,2017-11-11 00:40:49.412201+00,13,6333686.0,398655,approved,0,0,0,0,0,0.000000,6,6
1804842,6333880,0.000000,"""Yemen is a mess of their making"" -do you mean...",0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:42:17.154007+00,54,6332941.0,399491,approved,0,0,0,0,0,0.000000,0,4
1804843,6333885,0.300000,They may be wimps but not sure how many people...,0.000000,0.100000,0.000000,0.200000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-11 00:42:37.265238+00,55,6333627.0,399503,approved,0,0,0,0,0,0.000000,0,10


In [ ]:
df = df.drop_duplicates(subset={'comment_text'},keep='first')
df.shape

(1780823, 45)

In [ ]:

# Ref: https://www.kaggle.com/haqishen/jigsaw-predict
# We are creating a dict with shortened word as key and actual word as value.
apostophe_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                  "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                  "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is",
                  "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would",
                  "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have",
                  "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
                  "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
                  "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                  "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                  "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                  "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                  "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                  "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                  "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                  "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                  "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                  "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                  "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
                  "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                  "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                  "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is",
                  "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                  "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                  "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not",
                  "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                  "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
                  "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are",
                  "you've": "you have" }


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re

def process_sent(sent):
    line = ''
    for wrd in sent.split():
        if(len(apostophe_dict.get(wrd.lower(),'n'))>1 and wrd.lower() not in stopwords and len(wrd)>2):
            wrd = apostophe_dict[wrd.lower()]
            n_wrd = ''
            for w in wrd.split():
                if(w not in stopwords):
                    n_wrd += " "+w
            line += " "+n_wrd
        else:
            line += " "+ wrd.lower()    
    line = re.sub(r'[^a-zA-Z* ]',' ',line)
    return line

In [ ]:
# Preprocessing the comments.
preprocessed_data = []
from tqdm import tqdm
for sent in tqdm(df['comment_text'].values):
    sent = process_sent(sent)
    line = ''
    for wrd in sent.split():
        if(len(wrd)>2):
            line += " " +wrd.lower()
    line = re.sub(r"[']",'',line)
    preprocessed_data.append(line)

100%|██████████| 1780823/1780823 [01:52<00:00, 15786.86it/s]


In [ ]:
df['comment_text'] = preprocessed_data
len(df)

1780823

In [ ]:
# target with value greater than equal to 0.5 will be assigned 1 and rest 0.
def fun(x):
    if(x>=0.5):
        return 1
    else:
        return 0

In [ ]:
labels = df['target']
Y = labels.map(fun)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(preprocessed_data,Y,test_size=0.1,random_state=42)
# X_train,X_cv,Y_train,Y_cv = train_test_split(X_train,Y_train,test_size=0.1,random_state=42)
print(len(Y_test),len(X_test))

178083 178083


In [ ]:

encoded_essay1 = tokenizer.texts_to_sequences(X_test)
# defining a max size for padding.
max_len = 150
# padding the vectors of each datapoint to fixed length of 600.
test_sequence1 = pad_sequences(encoded_essay1,maxlen = max_len,padding='post')

In [ ]:
scores = model_dwn.evaluate(test_sequence1, Y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Test loss: 0.17635615958405979
Test accuracy: 0.9431725740432739


In [ ]:
scores

In [ ]:
# Roc auc
predicted_y = model_dwn.predict(test_sequence1,batch_size=128)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_test,predicted_y))


0.9290815908256576


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
# fpr, tpr, _ = metrics.roc_curve(Y_test,  predicted_y)
auc = metrics.roc_auc_score(Y_test, predicted_y)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()